In [62]:
import math
import pandas as pd
import numpy
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [91]:
#DATA CLEANING, PREPROCESSING
msg = pd.read_csv('C:/Users/RohanK/Documents/prog/news.csv', sep = ',', names = ['title', 'content', 'published_at', 'source', 'topic'])

nltk.download('stopwords')
lm = WordNetLemmatizer()
corpus = []

#for loop to iterarate over the data set
for i in range(0, len(msg)):
    review = re.sub('[^a-zA-Z0-9]', ' ', str(msg['content'][i]))
    review = review.lower()
    review = review.split()
    review = [lm.lemmatize(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RohanK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
#TF calculation
def tf(doc_list):
    #combined_str returns a single string with all the sentences in the doc
    combined_str = ' '.join(doc_list)
    #wordset returns all the unique words in combined_str
    wordset = [word for word in set(combined_str.split()) if len(word) > 1]
    
    filtered = []
    for document in doc_list:
        filtered.append(' '.join([word for word in document.split() if len(word) > 1]))
    
    #iterate through filtered list and calculating TF by dividing no. of times each word appears by the length of the doc
    doc_word_cnt = []
    for document in filtered:
        doc_word_cnt.append([document.split().count(word)/len(document.split()) for word in wordset])
    
    #returns dict with content as key and TF as value
    return{doc:term for doc,term in zip(filtered,doc_word_cnt)}

In [65]:
#function to return a single string with all the sentences in the doc
def word_list(doc_list):
    combined_str = ' '.join(doc_list)
    return[word for word in set(combined_str.split()) if len(word) > 1]

In [66]:
#Splitting the corpus into 90% train set and 10% test set using sklearn train_test_split
X_train, X_test = train_test_split(corpus, test_size = 0.10, random_state = 0)

In [67]:
#data frame that contains TF data using train set
#data contains values of the dict returned by function tf()
tf_df = pd.DataFrame(data = tf(X_train).values(),
                  columns = word_list(X_train),
                  index = tf(X_train).keys())
tf_df

,26th,prfctpatches,melody,city,button,5000,shetty,sustained,hurt,denial,...,1533409707762667527,bt,ikon,restaurant,coolest,stability,cute,presentation,exuding,voyage
bts member taehyung suga rm jin hope jimin jungkook making news every day various reason solo track group project bts may taken break seems like le break hope stunning everyone hair extension korea cover winning heart arson teaser bts aka kim taehyung suga aka min yoongi grabbed headline adorable video taegi bond also rumour taehyung suga flying japan video bts aka bangtan boy taehyung suga going viral army shared edited video two member popular boyband seen bonding food suga taehyung seen feeding sharing food one heart warming video ever one clip compiled bts army see suga giving bigger piece eatable bts army currently tripping suga taehyung bond video going viral everyone crushing taehyung suga feeding looking also rumour suga aka min yoongi kim taehyung flying japan taehyung suga daegu kid tom jerry ish bond suga physically clingy kind taehyung shower affection physical gesture hug holding hand chin flick misconception get along video prof otherwise suga working something detail yet surprising kind festa dinner jungkook however confirmed suga working something moreover hint suga working jimin project nothing confirmed yet taetae seen bonding wooga squad park seojoon peakboy park hyunsik choi woo shik friendcation soop teaser poster soop friendcation already soop friendcation streaming disney plus hotstar 22nd january every friday 11 pm kst going four episode soop friendcation also working album taehyung drive vlog revealed scraped lot song felt good enough meanwhile rumour taehyung suga soon flying japan however nothing confirmed also rumour jungkook flying schedule,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
come becoming avid listener pop term inadvertently end getting added vocabulary one term liner term talk year one born born 1997 example referred 97 liner recent year noted online netizens many idol currently receiving immense love enjoying popularity happen 97 liner member famous 97 liner squad bts jungkook seventeen mingyu the8 dk nct jaehyun got7 yugyeom bambam astro cha eun woo equally notable name like stray kid dependable leader oldest member bang chan year 1997 definitely brought absolute gem favourite male 97 liner pop idol take part poll share participate poll share pick http poll tryinteract com 62b02287ab395700182f60c3 method iframejoin biggest community pop fan live pinkvilla room get one step closer favourite celebs click join also read red velvet joy top june girl group member brand reputation ranking top 10 list inside,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
reaching hotel city rm revealed stay would four day added would step dinner sat roadside open air restaurant rm feasted beer burger fry said starving right grab food much quieter expected feel like rural town like familiar atmosphere rm attended art basel explained camera detail art fair also gave glimpse noodle beer followed soup noodle wrap showing pattern ping pong table rm said table look like bts symbol also spoke art piece viewed rm took tram visit foundation beyeler museum later took walk city third day rm visited kunstmuseum basel vitra design museum gallery walked around rm showed chair fan said breaking news guy coldplay chris martin made chair displayed vitra design museum see chris give call amazing rm next visited lucerne hiked mount rigi recalling previous visit lucerne rm added remember day crossing bridge buying souvenir also reminded bon voyage reality show featuring bts member rm jin suga hope jimin jungkook speaking camera rm said rode ssb train lucerne rode boat rode mountain train walked track road rode cable car boat planning go ride ssb rm travel switzerland ended visit museum tinguely next rm flew paris attend pinault collection visit musee ors

In [68]:
#IDF calculation
def idf(doc_list, unique_words):
    idf_dic = {}
    #compares uniqque words with content paras where if word in temp list matches idf_dic key, the value of the key increases by 1
    for word in unique_words:
        temp_list = []
        for sentence in doc_list:
            temp_list.append(word in sentence)
        idf_dic[word] = temp_list.count(True)+1
    
    #idf_list contains calculated IDF values
    idf_list = []
    for value in idf_dic.values():
        idf_list.append(math.log((len(tf_df)+1)/value)+1)
        
    return idf_list

In [69]:
#Calling idf() and passing content, unique words as parameters
idf_array = idf(tf_df.index,tf_df.columns)
idf_array

[5.799914262780603,
 6.898526551448713,
 5.026724374547121,
 3.830473616315096,
 5.394449154672439,
 6.898526551448713,
 6.898526551448713,
 6.898526551448713,
 4.333577193987177,
 6.898526551448713,
 6.898526551448713,
 3.763032335519563,
 1.163568459324062,
 3.260940391722327,
 4.595941458454668,
 6.898526551448713,
 3.1728331242120604,
 6.898526551448713,
 5.645763582953345,
 5.982235819574558,
 6.898526551448713,
 3.3721660268325513,
 4.125937829208931,
 6.898526551448713,
 5.799914262780603,
 6.898526551448713,
 6.898526551448713,
 3.4020189899822326,
 3.6998534338980313,
 6.898526551448713,
 6.898526551448713,
 6.493061443340548,
 6.898526551448713,
 6.898526551448713,
 6.898526551448713,
 6.898526551448713,
 6.2053793708887675,
 6.2053793708887675,
 5.799914262780603,
 3.7415261302985994,
 5.289088639014612,
 6.898526551448713,
 5.799914262780603,
 6.493061443340548,
 6.898526551448713,
 5.394449154672439,
 6.898526551448713,
 4.333577193987177,
 6.493061443340548,
 5.7999142627

In [70]:
#TF-IDF calculation
def tfidf(tf_data, idf_data):
    tfidf_list = []
    multiplied_matrix = numpy.array(tf_data) * numpy.array(idf_data)
    
    for array in multiplied_matrix:
        #dividing array by norm of array
        tfidf_list.append(array/(sum(array**2))**0.5)
        
    return tfidf_list

In [71]:
#data frame that contains TF-IDF data using train set
tfidf_df = pd.DataFrame(data = tfidf(tf_df.to_numpy(), numpy.array(idf_array)),
             columns = word_list(X_train),
             index = tf(X_train).keys())

tfidf_df

,26th,prfctpatches,melody,city,button,5000,shetty,sustained,hurt,denial,...,1533409707762667527,bt,ikon,restaurant,coolest,stability,cute,presentation,exuding,voyage
bts member taehyung suga rm jin hope jimin jungkook making news every day various reason solo track group project bts may taken break seems like le break hope stunning everyone hair extension korea cover winning heart arson teaser bts aka kim taehyung suga aka min yoongi grabbed headline adorable video taegi bond also rumour taehyung suga flying japan video bts aka bangtan boy taehyung suga going viral army shared edited video two member popular boyband seen bonding food suga taehyung seen feeding sharing food one heart warming video ever one clip compiled bts army see suga giving bigger piece eatable bts army currently tripping suga taehyung bond video going viral everyone crushing taehyung suga feeding looking also rumour suga aka min yoongi kim taehyung flying japan taehyung suga daegu kid tom jerry ish bond suga physically clingy kind taehyung shower affection physical gesture hug holding hand chin flick misconception get along video prof otherwise suga working something detail yet surprising kind festa dinner jungkook however confirmed suga working something moreover hint suga working jimin project nothing confirmed yet taetae seen bonding wooga squad park seojoon peakboy park hyunsik choi woo shik friendcation soop teaser poster soop friendcation already soop friendcation streaming disney plus hotstar 22nd january every friday 11 pm kst going four episode soop friendcation also working album taehyung drive vlog revealed scraped lot song felt good enough meanwhile rumour taehyung suga soon flying japan however nothing confirmed also rumour jungkook flying schedule,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
come becoming avid listener pop term inadvertently end getting added vocabulary one term liner term talk year one born born 1997 example referred 97 liner recent year noted online netizens many idol currently receiving immense love enjoying popularity happen 97 liner member famous 97 liner squad bts jungkook seventeen mingyu the8 dk nct jaehyun got7 yugyeom bambam astro cha eun woo equally notable name like stray kid dependable leader oldest member bang chan year 1997 definitely brought absolute gem favourite male 97 liner pop idol take part poll share participate poll share pick http poll tryinteract com 62b02287ab395700182f60c3 method iframejoin biggest community pop fan live pinkvilla room get one step closer favourite celebs click join also read red velvet joy top june girl group member brand reputation ranking top 10 list inside,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
reaching hotel city rm revealed stay would four day added would step dinner sat roadside open air restaurant rm feasted beer burger fry said starving right grab food much quieter expected feel like rural town like familiar atmosphere rm attended art basel explained camera detail art fair also gave glimpse noodle beer followed soup noodle wrap showing pattern ping pong table rm said table look like bts symbol also spoke art piece viewed rm took tram visit foundation beyeler museum later took walk city third day rm visited kunstmuseum basel vitra design museum gallery walked around rm showed chair fan said breaking news guy coldplay chris martin made chair displayed vitra design museum see chris give call amazing rm next visited lucerne hiked mount rigi recalling previous visit lucerne rm added remember day crossing bridge buying souvenir also reminded bon voyage reality show featuring bts member rm jin suga hope jimin jungkook speaking camera rm said rode ssb train lucerne rode boat rode mountain train walked track road rode cable car boat planning go ride ssb rm travel switzerland ended visit museum tinguely next rm flew paris attend pinault collection visit musee o

In [75]:
#TF-IDF OF TEST SET
#TF calculation using test set
tf_df_test = pd.DataFrame(data = tf(X_test).values(),
                  columns = word_list(X_test),
                  index = tf(X_test).keys())
tf_df_test

,comment,foundation,melody,city,atlanta,button,climbed,manage,temper,intrigue,...,restaurant,cute,blessed,entering,laurent,registration,design,browsing,delighted,twerked
sm entertainment management company nct announced january 6th 2023 haechan taking part upcoming nct 127 tour major health issue reported missing group rsquo activity nbsp rsquo know health tour nct haechan rsquo health issue member pop boy band nct dream named haechan recently promoted nct dream following release new remake song quot candy quot took sabbatical activity health issue notice released jan sm entertainment nct rsquo agency said haechan miss scheduled event tour pertaining poor health nbsp back haechan encountered unusual symptom including chest tightness heart palpitation manager drove emergency department underwent consultation examination given diagnosis needing treatment rest required haechan said participate planned event 39 present latin american leg nct 127 39 quot neo city link quot concert tour begin january run january 28 haechan 39 health rehabilitation utmost importance wish get well sooner resting decided abstain planned event starting week soon confirms return agency promise inform follower also genuinely regret inconvenience caused fan beg understanding choice necessary haechan 39 well agency pledge whatever necessary work hard help haechan may meet fan healthier image date tour band nct kickstarted world tour back october 2022 release fourth studio album baddie nct 127 performed many place like la jakarta bangkok korea etc following enthusiasm nct 127 decided excite fan united state nct 127 revealed nine additional date forthcoming neo city link globe tour december 2022 seven location including chicago houston united state atilde paulo santiago latin america host performance pop boy band pop superstar perform revised date january ndash january 28 2023 exact city lineup according poster three gig scheduled chicago january houston january 11 atlanta january 13 nct 127 next go latin america perform six performance four different location begin performing sao paulo brazil january 18 20 move santiago chile january 22 bogota colombia january 25 mexico january 28 lineup conclude nct 39 largest tour america band 39 follower eagerly anticipating amazing live performance haechan miss nct 127 39 tour latin america haechan notified would take stage next nct 127 latin american tour date obvious health issue member absent schedule illness first time nct encountered situation jaehyun issue mark lee also spoken exhaustion group rsquo comeback attend haechan 39 absence owing sickness mean nct 127 play tour fewer member member mdash yuta nakamoto jaeyun taeyong jungwoo mark lee winwin doyoung johnny taeil mdash take part usa latin america performance planned one loved sub unit sm entertainment group nct ticket usually get quickly sold hopefully fan finally get see least member end month minus haechan certain concert still wonderful even without haechan member present nct 127 rsquo much needed comeback thrilled band nct 127 make comeback tour begun repackaged album ay yo released later month said haechan presently taking care hoping soon feel well hope meet fan get back band soon wish quick recovery also read pop sensation rise nct 127 nct dream record breaking album sale,0.000000,0.0,0.000000,0.006303,0.002101,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
video start screen phone shown jungkook message flashing song start message hi something say ahead dropping track jungkook took twitter shared screenshot text message hi bunny emoji written captioned post 22 06 13 00 00 reacting song fan wrote like poured emotion song seriously butterfly dancing stomach voice soo soothing another one wrote jungkook idol artist care love much like army feel appreciated one fan also commented song soo cute look clearly express love army one comment read jungkook voice beautiful song masterpiece comment read jungkook gem precious anything gli

In [76]:
#idf calculation using test set
idf_array_test = idf(tf_df_test.index,tf_df_test.columns)
idf_array_test

[2.0394234172777788,
 4.713572066704308,
 3.614959778036198,
 3.46080909820894,
 4.713572066704308,
 4.308106958596143,
 4.713572066704308,
 3.327277705584417,
 4.308106958596143,
 4.713572066704308,
 3.797281334830153,
 4.0204248861443626,
 4.0204248861443626,
 3.209494669928034,
 4.0204248861443626,
 3.0088239744658827,
 4.713572066704308,
 3.797281334830153,
 4.308106958596143,
 2.0394234172777788,
 4.713572066704308,
 3.327277705584417,
 4.0204248861443626,
 4.713572066704308,
 4.308106958596143,
 4.713572066704308,
 4.308106958596143,
 4.713572066704308,
 4.713572066704308,
 4.713572066704308,
 3.327277705584417,
 4.308106958596143,
 4.308106958596143,
 4.713572066704308,
 4.713572066704308,
 4.308106958596143,
 3.327277705584417,
 3.209494669928034,
 3.46080909820894,
 3.797281334830153,
 4.308106958596143,
 4.713572066704308,
 4.713572066704308,
 4.713572066704308,
 4.308106958596143,
 4.713572066704308,
 4.308106958596143,
 4.713572066704308,
 4.713572066704308,
 3.008823974465

In [78]:
#TFIDF calculation using test set
tfidf_df_test = pd.DataFrame(data = tfidf(tf_df_test.to_numpy(), numpy.array(idf_array_test)),
             columns = word_list(X_test),
             index = tf(X_test).keys())

tfidf_df_test

,comment,foundation,melody,city,atlanta,button,climbed,manage,temper,intrigue,...,restaurant,cute,blessed,entering,laurent,registration,design,browsing,delighted,twerked
sm entertainment management company nct announced january 6th 2023 haechan taking part upcoming nct 127 tour major health issue reported missing group rsquo activity nbsp rsquo know health tour nct haechan rsquo health issue member pop boy band nct dream named haechan recently promoted nct dream following release new remake song quot candy quot took sabbatical activity health issue notice released jan sm entertainment nct rsquo agency said haechan miss scheduled event tour pertaining poor health nbsp back haechan encountered unusual symptom including chest tightness heart palpitation manager drove emergency department underwent consultation examination given diagnosis needing treatment rest required haechan said participate planned event 39 present latin american leg nct 127 39 quot neo city link quot concert tour begin january run january 28 haechan 39 health rehabilitation utmost importance wish get well sooner resting decided abstain planned event starting week soon confirms return agency promise inform follower also genuinely regret inconvenience caused fan beg understanding choice necessary haechan 39 well agency pledge whatever necessary work hard help haechan may meet fan healthier image date tour band nct kickstarted world tour back october 2022 release fourth studio album baddie nct 127 performed many place like la jakarta bangkok korea etc following enthusiasm nct 127 decided excite fan united state nct 127 revealed nine additional date forthcoming neo city link globe tour december 2022 seven location including chicago houston united state atilde paulo santiago latin america host performance pop boy band pop superstar perform revised date january ndash january 28 2023 exact city lineup according poster three gig scheduled chicago january houston january 11 atlanta january 13 nct 127 next go latin america perform six performance four different location begin performing sao paulo brazil january 18 20 move santiago chile january 22 bogota colombia january 25 mexico january 28 lineup conclude nct 39 largest tour america band 39 follower eagerly anticipating amazing live performance haechan miss nct 127 39 tour latin america haechan notified would take stage next nct 127 latin american tour date obvious health issue member absent schedule illness first time nct encountered situation jaehyun issue mark lee also spoken exhaustion group rsquo comeback attend haechan 39 absence owing sickness mean nct 127 play tour fewer member member mdash yuta nakamoto jaeyun taeyong jungwoo mark lee winwin doyoung johnny taeil mdash take part usa latin america performance planned one loved sub unit sm entertainment group nct ticket usually get quickly sold hopefully fan finally get see least member end month minus haechan certain concert still wonderful even without haechan member present nct 127 rsquo much needed comeback thrilled band nct 127 make comeback tour begun repackaged album ay yo released later month said haechan presently taking care hoping soon feel well hope meet fan get back band soon wish quick recovery also read pop sensation rise nct 127 nct dream record breaking album sale,0.000000,0.0,0.000000,0.067967,0.030857,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
video start screen phone shown jungkook message flashing song start message hi something say ahead dropping track jungkook took twitter shared screenshot text message hi bunny emoji written captioned post 22 06 13 00 00 reacting song fan wrote like poured emotion song seriously butterfly dancing stomach voice soo soothing another one wrote jungkook idol artist care love much like army feel appreciated one fan also commented song soo cute look clearly express love army one comment read jungkook voice beautiful song masterpiece comment read jungkook gem precious anything gli